In [ ]:
# Install package for Japanese dates using pip install erajp


In [1]:
#!/usr/bin/env python
import tika
tika.initVM()
from tika import parser

In [281]:
parsed = parser.from_file("sample_ids_in/Sample1.pdf")
data= parsed["content"]
# with open('Sample1_extracted.txt', 'w', encoding="utf-8") as f:
#         f.write("%s" % data)

In [292]:
parsed = parser.from_file("sample_ids_in/Sample2.pdf")
data= parsed["content"]
# with open('Sample2_extracted.txt', 'w', encoding="utf-8") as f:
#         f.write("%s" % data)

In [ ]:
# parsed = parser.from_file("sample_ids_in/Sample3.pdf")
# data= parsed["content"]
# with open('Sample3_extracted.txt', 'w', encoding="utf-8") as f:
#         f.write("%s" % data)

# Extracts item 1: account_closing_month

In [282]:
import re, datetime  

# re.findall('から(.*)まで', data) # to find all instances of date
result = re.search('から(.*)まで', data)
s = result.group(1)
date = [int(s) for s in re.findall(r'\d+', s)]
date = datetime.datetime(date[0], date[1], date[2])
alpha = date.year
if date.year == 30:
    alpha = 2018  
account_closing_month = f"{alpha}{date.month:02d}"
account_closing_month


'201803'

# Extract item 2: account_standard_type
"連結貸借対照表" = 0

"連結財政状態計算書" = 1

In [283]:
import re
result1 = re.search('連結貸借対照表', data)
result2 = re.search('連結財政状態計算書', data)
if result1 is None and result2 is not None:
    account_standard_type = 1
elif result1 is not None and result2 is None:
        account_standard_type = 0
else:
    account_standard_type = "Something is wrong, may be check if PDF is correct"

        
print(f"The account standard type is {account_standard_type}")

The account standard type is 0


<_sre.SRE_Match object; span=(44742, 44749), match='連結貸借対照表'>

# Extract item 3: hakkoukano_kabu

In [293]:
result = re.search('発行可能株式総数(.*)株', data)
hakkoukano_kabu = result.group(1).strip().replace(',', '')
print(f"The hakkoukano_kabu is {hakkoukano_kabu}")

The hakkoukano_kabu is 280000000


# Extract item 4 and 5: hakkouzumi_kabu_c

In [294]:
hakkoukano_kabu_c = None
hakkoukano_kabu_nc = None


result1 = re.search('発行済株式の総数(.*)株', data)
result2 = re.search('自己株式(.*)株', data)

if result1 is None:
    print("Something is wrong")
else:
    if result2 is not None:
        print("It is consolidated")
        #hakkoukano_kabu_c = result1.group(1).strip().replace(',', '') # works with sample 1
        hakkoukano_kabu_c = re.findall('\d+', result1.group(1).replace(',', ''))[0] #changed due to sample 2

    elif result2 is None:
        print("It is unconsolidated")
        hakkoukano_kabu_nc =  result1.group(1).strip().replace(',', '')
    else:
        print("Something is wrong")
    
print(f"The hakkoukano_kabu_c is {hakkoukano_kabu_c}")
print(f"The hakkoukano_kabu_nc is {hakkoukano_kabu_nc}")
 

It is consolidated
The hakkoukano_kabu_c is 87789098
The hakkoukano_kabu_nc is None


In [271]:
result1

<_sre.SRE_Match object; span=(31039, 31075), match='発行済株式の総数 87,789,098株（自己株式 2,096,634株'>

# Extract item 6: jikokabu

In [297]:
# works for both sample1 and sample2

result1 = re.search('発行済株式の総数(.*)株', data)
start = result1.span()[0]
sub_data = data[start: start + 50]

result = re.search('自己株式(.*)株', sub_data)
jikokabu = result.group(1).strip().replace(',', '')
print(f"The jikokabu is {jikokabu}")

###works for sample1
# result = re.search('自己株式(.*)株', data)
# jikokabu = result.group(1).strip().replace(',', '')
# print(f"The jikokabu is {jikokabu}")

The jikokabu is 2096634


# Extract the consolidated balance sheet data and yen multiplying factor

In [296]:
# Basically, I tried to extract the string between 連結貸借対照表 iten 2 and "非支配株主持分 item 24. Many of the strings are inside that area. 

## result = re.search('連結貸借対照表(.*)"非支配株主持分', data) #it seems that re.search does not work for such a long area

start = data.index('連結貸借対照表')
end   = data.index("非支配株主持分")
consolidated_data = data[start: end+20] #take a little extra because we want value of 非支配株主持分 
## re.sub(r"\s+", "", consolidated_data) # remove all white spaces
# Task To find the （単位：百万円), so that multiplying factor can be determined. 
# The approach is the following in order

# 1. pat = (something), string = consolidate_data
# 2. pat = split string by :
# 3. Take the remaining of the spplited string

#result1 = re.findall('\(.*?\)',consolidated_data) #seems this does not work because, brackets are not ()
idx_result1 = consolidated_data.index("単位")
result2 = consolidated_data[idx_result1: idx_result1+6]
if "百万円" in result2:
    mult_int = 1000000
elif "千円" in result2:
    mult_int = 1000
else: 
    print ("Unit is missing. Something Wrong")

print(f"The multiply factor is {mult_int}")

ValueError: substring not found

In [316]:
start = data.index('連 結 貸 借 対 照 表'); start

ValueError: substring not found

In [300]:
start = data.index('連結貸借対照表'); start

44742

In [315]:
start = re.search(r'\b連結貸借対照表\b', data);  
idx = start.span()[0]; idx

44742

In [319]:
start = re.search(r'\b連　結　貸　借　対　照　表\b', data);  
idx = start.span()[0]; idx

43605

In [331]:
print(data[idx: idx + 1000])

連　結　貸　借　対　照　表
（平成30年３月31日現在）

（単位：百万円）
科 目 金 額 科 目 金 額

（資産の部） （負債の部）
流 動 資 産

現 金 及 び 預 金
受 取 手 形 及 び 売 掛 金
リース債権及びリース投資資産
有 価 証 券
た な 卸 資 産
繰 延 税 金 資 産
そ の 他
貸 倒 引 当 金

固 定 資 産
有 形 固 定 資 産

建 物 及 び 構 築 物
機 械 装 置 及 び 運 搬 具
土 地
リ ー ス 資 産
そ の 他

無 形 固 定 資 産
ソ フ ト ウ ェ ア
ソ フ ト ウ ェ ア 仮 勘 定
の れ ん
そ の 他

投 資 そ の 他 の 資 産
投 資 有 価 証 券
退 職 給 付 に 係 る 資 産
繰 延 税 金 資 産
そ の 他
貸 倒 引 当 金

168,670
38,032
94,438
4,747

100
9,222
6,606

15,884
△360

200,833
75,680
39,990
7,547

19,373
2,952
5,815

18,915
12,506
4,359

707
1,341

106,238
78,766
5,610
4,902

17,230
△271

流 動 負 債 81,312
支 払 手 形 及 び 買 掛 金 23,246
短 期 借 入 金 4,460
未 払 法 人 税 等 6,139
賞 与 引 当 金 13,184
受 注 損 失 引 当 金 1,075
そ の 他 引 当 金 72
そ の 他 33,133

固 定 負 債 61,893
長 期 借 入 金 25,482
リ ー ス 債 務 3,997
役 員 退 職 慰 労 引 当 金 65
退 職 給 付 に 係 る 負 債 17,296
繰 延 税 金 負 債 6,473
再評価に係る繰延税金負債 403
そ の 他 8,173
負 債 合 計 143,205

（純資産の部）
株 主 資 本 193,941

資 本 金 10,001
資 本 剰 余 金 83,384
利 益 剰 余 金 108,298
自 己 株 式 △7,742

そ の 他 の 包 括 利 益 累 計 額 27,692
その他有価証券評価差額金 31,922
土 地 再 評 

In [330]:
a="連　結　貸　借　対　照　表"; a

'連\u3000結\u3000貸\u3000借\u3000対\u3000照\u3000表'

In [333]:
b = "連　結　貸　借　対　照　表"; b


'連\u3000結\u3000貸\u3000借\u3000対\u3000照\u3000表'

In [332]:
a="連 結 貸 借 対 照 表"; a

'連 結 貸 借 対 照 表'

In [ ]:
a

## Extract item 7: kabunusi_sihon_ren

In [291]:
result1 = re.search("株主資本", consolidated_data)
sign = 1
if result1:
    index = result1.span()[1]
    s = consolidated_data[index: index + 15]
    s = s.replace(",", "")
    s = s.replace(" ","") 
    if "△" in s and s.index("△") == 0:
        sign = -1 
    else:
        sign = 1
else:
    print("something wrong")
    
result2 = int(re.findall('\d+', s)[0])
kabunusi_sihon_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

print(f"The kabunusi_sihon_ren is {kabunusi_sihon_ren}")

The kabunusi_sihon_ren is 77600000000


## Extract item 8: sihon_ren

In [180]:
result1 = re.search("資本金", consolidated_data)
sign = 1
if result1:
    index = result1.span()[1]
    s = consolidated_data[index: index + 15]
    s = s.replace(",", "")
    s = s.replace(" ","") 
    if "△" in s and s.index("△") == 0:
        sign = -1 
    else:
        sign = 1
else:
    print("something wrong")
    
result2 = int(re.findall('\d+', s)[0])
sihon_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

print(f"The sihon_ren is {sihon_ren}")

The sihon_ren is 10000000000


# Extract item 9: sihon_jyoyo_ren

In [181]:
result1 = re.search("資本剰余金", consolidated_data)
sign = 1
if result1:
    index = result1.span()[1]
    s = consolidated_data[index: index + 15]
    s = s.replace(",", "")
    s = s.replace(" ","") 
    if "△" in s and s.index("△") == 0:
        sign = -1 
    else:
        sign = 1
else:
    print("something wrong")
    
result2 = int(re.findall('\d+', s)[0])
sihon_jyoyo_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

print(f"The sihon_jyoyo_ren is {sihon_jyoyo_ren}")

The sihon_jyoyo_ren is 38285000000


## Extract item 10: rieki_jyoyo_ren

In [182]:
result1 = re.search("利益剰余金", consolidated_data)
sign = 1
sign = 1
if result1:
    index = result1.span()[1]
    s = consolidated_data[index: index + 15]
    s = s.replace(",", "")
    s = s.replace(" ","") 
    if "△" in s and s.index("△") == 0:
        sign = -1 
    else:
        sign = 1
else:
    print("something wrong")
    
result2 = int(re.findall('\d+', s)[0])
rieki_jyoyo_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

print(f"The rieki_jyoyo_ren is {rieki_jyoyo_ren}")

The rieki_jyoyo_ren is 29353000000


## Extract item 11: jikokabu_ren

In [183]:
result1 = re.search("自己株式", consolidated_data)
sign = 1
if result1:
    index = result1.span()[1]
    s = consolidated_data[index: index + 15]
    s = s.replace(",", "")
    s = s.replace(" ","") 
    if "△" in s and s.index("△") == 0:
        sign = -1 
    else:
        sign = 1
else:
    print("something wrong")
    
result2 = int(re.findall('\d+', s)[0])
jikokabu_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

print(f"The jikokabu_ren is {jikokabu_ren}")

The jikokabu_ren is -38000000


## Extract item 12: sonota_hokatu_ren

In [184]:
result1_1 = re.search("その他の包括利益累計額", consolidated_data)
result1_2 = re.search("その他の資本の構成要素", consolidated_data)
result1_3 = re.search("累積その他の包括利益", consolidated_data)
resultList = [result1_1, result1_2, result1_3]

if any(resultList) is not True:
    sonota_hokatu_ren = []
    print(f"The sonota_hokatu_ren is {result2}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]

    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    sonota_hokatu_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The sonota_hokatu_ren is {sonota_hokatu_ren}")

The sonota_hokatu_ren is -14848000000


## Extract item 15: BLUE 




In [185]:
result1_1 = re.search("現金及び預金", consolidated_data)
result1_2 = re.search("現金及び現金同等物", consolidated_data)

resultList = [result1_1]
item_15 = []
if any(resultList) is not True:
    item_15 = []
    print(f"The Item 15 is {item_15}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_15 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The Item 15 is {item_15}")

The Item 15 is 37316000000


## Extract item 19: BLUE

In [186]:
result1_1 = re.search("投資有価証券", consolidated_data)

resultList = [result1_1]
item_19 = []
if any(resultList) is not True:
    item_19 = []
    print(f"The Item 19 is {item_19}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_19 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The Item 19 is {item_19}")

The Item 19 is 8808000000


## Extract item 24: BLUE

In [187]:
result1_1 = re.search("非支配株主持分", consolidated_data)

resultList = [result1_1]
item_24 = []
if any(resultList) is not True:
    item_24 = []
    print(f"The Item 24 is {item_24}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_24 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The Item 24 is {item_24}")

The Item 24 is 3063000000


# Moving to next page

## Extract the un-consolidated balance sheet data and yen multiplying factor

In [188]:
# Basically, I tried to extract the string between 連結貸借対照表 iten 2 and "非支配株主持分 item 24. Many of the strings are inside that area. 

## result = re.search('連結貸借対照表(.*)"非支配株主持分', data) #it seems that re.search does not work for such a long area

matches = re.search(r'\b貸借対照表\b', data)
start = matches.span()[1]
end   = data.index("評価・換算差額等")
consolidated_data = data[start: end+20]

# Task To find the （単位：百万円), so that multiplying factor can be determined. 
# The approach is the following in order

# 1. pat = (something), string = consolidate_data
# 2. pat = split string by :
# 3. Take the remaining of the spplited string

#result1 = re.findall('\(.*?\)',consolidated_data) #seems this does not work because, brackets are not ()
idx_result1 = consolidated_data.index("単位")
result2 = consolidated_data[idx_result1: idx_result1+6]
if "百万円" in result2:
    mult_int = 1000000
elif "千円" in result2:
    mult_int = 1000
else: 
    print ("Unit is missing. Something Wrong")

print(f"The multiply factor is {mult_int}")

The multiply factor is 1000000


## Extract item 13: oya_kizoku_ren 

In [190]:
result1_1 = re.search("親会社の所有者に帰属する持分合計", consolidated_data)
result1_2 = re.search("当社株主に帰属する資本", consolidated_data)
result1_3 = re.search("親会社持分合計", consolidated_data)
result1_4 = re.search("親会社の所有者に帰属する持分", consolidated_data)
resultList = [result1_1, result1_2, result1_3, result1_4]
oya_kizoku_ren = []
if any(resultList) is not True:
    oya_kizoku_ren = []
    print(f"The oya_kizoku_ren is {oya_kizoku_ren}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    oya_kizoku_ren = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The jikokabu_ren is {oya_kizoku_ren}")

The oya_kizoku_ren is []


## Extract item 14: kabunusi_sihon_tan

In [191]:
result1_1 = re.search("株主資本", consolidated_data)
resultList = [result1_1]
kabunusi_sihon_tan = []
if any(resultList) is not True:
    kabunusi_sihon_tan = []
    print(f"The kabunusi_sihon_tan is {kabunusi_sihon_tan}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    kabunusi_sihon_tan = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The kabunusi_sihon_tan is {kabunusi_sihon_tan}")

The kabunusi_sihon_tan is 67388000000


## Extract item 16: BLUE

In [192]:
result1_1 = re.search("現金及び預金", consolidated_data)
result1_2 = re.search("現金及び現金同等物", consolidated_data)
resultList = [result1_1, result1_2]
item_16 = []
if any(resultList) is not True:
    item_16 = []
    print(f"The item_16 is {item_16}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_16 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The item_16 is {item_16}")

The item_16 is 10593000000


## Extract item 20: BLUE

In [193]:
result1_1 = re.search("投資有価証券", consolidated_data)
resultList = [result1_1]
item_20 = []
if any(resultList) is not True:
    item_20 = []
    print(f"The item_20 is {item_20}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_20 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The item_20 is {item_20}")

The item_20 is 6872000000


## Extract item 21: BLUE

In [195]:
result1_1 = re.search("評価・換算差額等", consolidated_data)
resultList = [result1_1]
item_21 = []
if any(resultList) is not True:
    item_21 = []
    print(f"The item_21 is {item_21}")
else:
    res = [i for i, val in enumerate(resultList) if val][0] 
    result1 = resultList[res]
    sign = 1
    if result1:
        index = result1.span()[1]
        s = consolidated_data[index: index + 15]
        s = s.replace(",", "")
        s = s.replace(" ","") 
        if "△" in s and s.index("△") == 0:
            sign = -1 
        else:
            sign = 1
    else:
        print("something wrong")

    result2 = int(re.findall('\d+', s)[0])
    item_21 = result2 * mult_int * sign  # to get the correct number with trailing zeros

    print(f"The item_21 is {item_21}")

The item_21 is 3748000000


In [196]:
consolidated_data

'（2018年３月31日現在） （単位：百万円）\n\u3000\n\n科 目 金 額\n資産の部\n流動資産 56,458\n現金及び預金 10,593\n受取手形 478\n売掛金 23,596\n商品及び製品 9,079\n仕掛品 1,604\n原材料及び貯蔵品 1,783\n前渡金 138\n前払費用 689\n短期貸付金 5,042\n未収入金 3,406\n繰延税金資産 1,158\nその他 728\n貸倒引当金 △1,840\n固定資産 149,976\n有形固定資産 27,579\n建物 6,345\n構築物 138\n機械及び装置 573\n車両運搬具 83\n工具、器具及び備品 2,087\n土地 17,529\n建設仮勘定 820\n無形固定資産 9,530\nソフトウェア 9,438\nその他 92\n投資その他の資産 112,865\n投資有価証券 6,872\n関係会社株式 93,035\n出資金 1\n関係会社出資金 4,049\n長期貸付金 81\n関係会社長期貸付金 565\n破産更生債権等 49\n長期前払費用 555\n前払年金費用 7,344\nその他 440\n貸倒引当金 △129\n\n資産合計 206,434\n\n科 目 金 額\n負債の部\n流動負債 83,477\n支払手形 644\n買掛金 17,377\n短期借入金 23,963\n一年内返済予定の長期借入金 23,798\nリース債務 40\n未払金 8,605\n未払費用 5,019\n未払法人税等 374\n前受金 1,204\n預り金 682\n製品保証引当金 227\n受注損失引当金 678\nその他 860\n固定負債 51,820\n長期借入金 37,059\nリース債務 60\n再評価に係る繰延税金負債 1,461\n繰延税金負債 3,228\n退職給付引当金 7,903\n関係会社事業損失引当金 1,361\n資産除去債務 536\nその他 209\n負債合計 135,297\n純資産の部\n株主資本 67,388\n資本金 10,000\n資本剰余金 48,494\n資本準備金 10,000\nその他資本剰余金 38,494\n利益剰余金 8,932\nその他利益剰余金 8,932\n繰越利益剰余金 8,932\n\n自己株式 △38\n評価

In [200]:

matches = re.search(r'\b貸借対照表\b', data)
start = matches.span()[1]
end   = data.index("評価・換算差額等")
consolidated_data = data[start: end+150]


In [201]:
consolidated_data

'（2018年３月31日現在） （単位：百万円）\n\u3000\n\n科 目 金 額\n資産の部\n流動資産 56,458\n現金及び預金 10,593\n受取手形 478\n売掛金 23,596\n商品及び製品 9,079\n仕掛品 1,604\n原材料及び貯蔵品 1,783\n前渡金 138\n前払費用 689\n短期貸付金 5,042\n未収入金 3,406\n繰延税金資産 1,158\nその他 728\n貸倒引当金 △1,840\n固定資産 149,976\n有形固定資産 27,579\n建物 6,345\n構築物 138\n機械及び装置 573\n車両運搬具 83\n工具、器具及び備品 2,087\n土地 17,529\n建設仮勘定 820\n無形固定資産 9,530\nソフトウェア 9,438\nその他 92\n投資その他の資産 112,865\n投資有価証券 6,872\n関係会社株式 93,035\n出資金 1\n関係会社出資金 4,049\n長期貸付金 81\n関係会社長期貸付金 565\n破産更生債権等 49\n長期前払費用 555\n前払年金費用 7,344\nその他 440\n貸倒引当金 △129\n\n資産合計 206,434\n\n科 目 金 額\n負債の部\n流動負債 83,477\n支払手形 644\n買掛金 17,377\n短期借入金 23,963\n一年内返済予定の長期借入金 23,798\nリース債務 40\n未払金 8,605\n未払費用 5,019\n未払法人税等 374\n前受金 1,204\n預り金 682\n製品保証引当金 227\n受注損失引当金 678\nその他 860\n固定負債 51,820\n長期借入金 37,059\nリース債務 60\n再評価に係る繰延税金負債 1,461\n繰延税金負債 3,228\n退職給付引当金 7,903\n関係会社事業損失引当金 1,361\n資産除去債務 536\nその他 209\n負債合計 135,297\n純資産の部\n株主資本 67,388\n資本金 10,000\n資本剰余金 48,494\n資本準備金 10,000\nその他資本剰余金 38,494\n利益剰余金 8,932\nその他利益剰余金 8,932\n繰越利益剰余金 8,932\n\n自己株式 △38\n評価

In [209]:
start = data.index('連結貸借対照表')
end   = data.index("非支配株主持分")
consolidated_data = data[start: end+250] #take a little extra because we want value of 非支配株主持分 
consolidated_data

'連結貸借対照表（2018年３月31日現在） （単位：百万円）\n\u3000\n\n科 目 金 額\n資産の部\n流動資産 155,182\n現金及び預金 37,316\n受取手形及び売掛金 59,789\n商品及び製品 28,664\n仕掛品 5,018\n原材料及び貯蔵品 12,914\n繰延税金資産 3,638\nその他 9,016\n貸倒引当金 △1,175\n固定資産 99,159\n有形固定資産 50,216\n建物及び構築物 12,582\n機械装置及び運搬具 8,592\n工具、器具及び備品 6,744\n土地 21,041\n建設仮勘定 1,255\n無形固定資産 17,757\nのれん 4,483\nソフトウェア 10,844\nその他 2,429\n投資その他の資産 31,185\n投資有価証券 8,808\n退職給付に係る資産 16,555\n繰延税金資産 3,105\nその他 3,207\n貸倒引当金 △491\n\n資産合計 254,342\n\n科 目 金 額\n負債の部\n流動負債 110,321\n支払手形及び買掛金 36,212\n短期借入金 4,429\n一年内返済予定の長期借入金 24,561\n未払金 10,943\n未払費用 19,448\n未払法人税等 1,631\n製品保証引当金 1,315\n返品調整引当金 1,431\n受注損失引当金 678\nその他 9,668\n固定負債 78,205\n長期借入金 38,388\n再評価に係る繰延税金負債 1,461\n繰延税金負債 8,127\n退職給付に係る負債 26,676\nその他 3,550\n負債合計 188,526\n純資産の部\n株主資本 77,600\n資本金 10,000\n資本剰余金 38,285\n利益剰余金 29,353\n自己株式 △38\nその他の包括利益累計額 △14,848\nその他有価証券評価差額金 1,305\n繰延ヘッジ損益 △588\n土地再評価差額金 3,316\n為替換算調整勘定 △16,203\n退職給付に係る調整累計額 △2,678\n非支配株主持分 3,063\n純資産合計 65,816\n負債及び純資産合計 254,342\n\n\u3000\n\n▶\n事業報告の「会社の体制および方針」、連結計算書類の連結株主資本等変

In [212]:
re.search("会社の体制および方",consolidated_data)

<_sre.SRE_Match object; span=(889, 898), match='会社の体制および方'>